In [52]:
import pandas as pd
import numpy as np
from IPython.display import display

### Enter the file path here

In [53]:
# Enter the entire file path
published_KT_file = r'C:\Users\Daraz\Documents\ddda\KB\2022_06290629_P_SKT.xlsx'
# disabled_KT_file = r'C:\Users\Daraz\Documents\ddda\KB\2022_06290629_P_SKT.xlsx'

In [54]:
df_published_entire = pd.read_excel(published_KT_file, sheet_name = "entire version",header=0,engine ="openpyxl")
# df_disabled_entire = pd.read_excel(disabled_KT_file, sheet_name = "entire version",header=0,engine ="openpyxl")

c:\Users\Daraz\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [55]:
display(df_published_entire.head(2))
# display(df_disabled_entire.head(2))

,Category id,Category,Knowledge id,Knowledge title,Knowledge type,Create time,Edit time,Editor,Similar question,机器人,AIML&Android问题&IOS问题&会员帮助&无线知识&机器人
0,1000002193,English/Payment/Payment,1000015107,How do I pay on Daraz,普通知识,2020-03-26 19:32:18,2022-06-23 18:25:28,md.abid.hassan,How do I make payment on Daraz\nPlease let me ...,How do I pay on Daraz?Daraz supports multiple ...,NaN
1,1000002193,English/Payment/Payment,1000015113,How do I pay with my Card,普通知识,2020-03-26 20:06:13,2022-01-05 20:08:37,md.abid.hassan,How can I make payment with my Debit card\nHow...,"After Checkout, when customers click on Procee...",NaN


In [56]:
display(df_published_entire.tail(2))
# display(df_disabled_entire.tail(2))

,Category id,Category,Knowledge id,Knowledge title,Knowledge type,Create time,Edit time,Editor,Similar question,机器人,AIML&Android问题&IOS问题&会员帮助&无线知识&机器人
524,1000005714,English/Pre match,1000068611,Need more help,普通知识,2022-03-22 18:26:37,2022-03-23 17:21:43,md.abid.hassan,need more help,id - 8346:2.0.0,NaN
525,1000005714,English/Pre match,1000070868,Aro shahajjo chai,普通知识,2022-04-17 16:24:08,2022-04-17 16:24:09,md.abid.hassan,aro shahajjo chai,id - 8603:2.0.0,NaN


In [57]:
df_published_entire['State'] = 'Published'
# df_disabled_entire['State'] = 'Disabled'

In [58]:
# entire_version_updated = pd.concat([df_published_entire,df_disabled_entire], axis=0)
entire_version_updated = df_published_entire

In [59]:
entire_version_updated['Language'] = entire_version_updated['Category'].apply(lambda x: x.split('-')[0].strip())

In [60]:
print('No SQ Found: ')
entire_version_updated[entire_version_updated['Similar question'].isna()]['Knowledge id']

No SQ Found: 


522    1000068572
Name: Knowledge id, dtype: int64

In [61]:
entire_version_updated['Similar question'].fillna('No SQ', inplace = True)

In [62]:
entire_version_updated['Similar question'].isna().sum()

0

In [63]:
similar_questions=[]

for i in range(len(entire_version_updated)):
        
        similar_questions.append(entire_version_updated["Similar question"].to_list()[i].split('\n')) #Create a list with SQs


In [64]:
knowledge_id_list = entire_version_updated["Knowledge id"].to_list()

In [65]:
df_SQ = pd.DataFrame(list(zip(similar_questions,knowledge_id_list)), columns=['SQ','KT_id'])

In [66]:
df_SQ.tail()

,SQ,KT_id
521,"[need to solve order error, resolve order plac...",1000067895
522,[No SQ],1000068572
523,"[common reasons for delivery delay, reason for...",1000068576
524,[need more help],1000068611
525,[aro shahajjo chai],1000070868


In [67]:
df_SQ_all = df_SQ.explode('SQ') ## Seperate all the SQs according to the KT ID

In [68]:
df_SQ_all

,SQ,KT_id
0,How do I make payment on Daraz,1000015107
0,Please let me know the payment procedure on Daraz,1000015107
0,Kindly let me know the payment process on Daraz,1000015107
0,Please tell me the payment system on Daraz.,1000015107
0,What is the system to make payment on Daraz,1000015107
...,...,...
523,Tell me why delivery gets delay usually,1000068576
523,Tell me the reason of delay delivery,1000068576
523,What are the foremost common reasons for delay...,1000068576
524,need more help,1000068611


In [69]:
df_ = df_SQ_all.groupby(by='SQ').agg({'KT_id': lambda x: x.tolist()}).reset_index() ## Implode the dataframe by the SQs.

In [70]:
df_

,SQ,KT_id
0,,"[1000017470, 1000017485, 1000017496, 100001751..."
1,Any Home shifting services on Daraz,[1000017637]
2,Any Tatto service on Daraz,[1000017662]
3,Are you guys providing Home shifting service,[1000017637]
4,EMI payment method which Visa Debit Card is ...,[1000015635]
...,...,...
7803,your delivery man intentionally delaying the d...,[1000015453]
7804,your hub time,[1000015535]
7805,your return policy what,[1000016013]
7806,your worst service earned a review in e commer...,[1000020671]


In [71]:
df_['Count'] = df_['KT_id'].apply(lambda x : len(x))

In [72]:
df_sq_duplicates =  df_[df_['Count'] > 1]

In [73]:
df_sq_ = df_sq_duplicates.reset_index()

In [74]:
df_sq_

,index,SQ,KT_id,Count
0,0,,"[1000017470, 1000017485, 1000017496, 100001751...",9
1,98,3rd item from top in the screenshot isn’t deli...,"[1000015482, 1000017233]",2
2,221,Any chance to change my phone number of my giv...,"[1000015446, 1000021471]",2
3,283,Any update about my order delivery,"[1000015451, 1000017233]",2
4,350,At present in which areas are you offering Sp...,"[1000017508, 1000017513]",2
...,...,...,...,...
72,7415,there I've place an order but I've got no resp...,"[1000015246, 1000015249]",2
73,7727,why delivery fail,"[1000017218, 1000017220]",2
74,7730,why didn’t you inform that you are unable to d...,"[1000017218, 1000017220]",2
75,7745,why my delivery is cancelled,"[1000017218, 1000017220]",2


In [75]:
df_sq_[['Status', 'Comment']] = np.nan

In [76]:
df_sq_.drop(labels='index', axis = 1, inplace = True)

In [77]:
df_sq_.to_clipboard()

In [78]:
df_sq_.to_excel('SQ_repetition_check_results.xlsx')

## Previous code

In [79]:

# def check_for_duplicate_SQ(entire_version_updated):
#     knowledge_id_list = entire_version_updated["Knowledge id"].to_list()
#     similar_questions=[]
#     for i in range(0,len(entire_version_updated)):
#         try:
#             similar_questions.append(entire_version_updated["Similar question"].to_list()[i].split('\n'))
#         except:
#             print("Empty SQ field detected for " + str(entire_version_updated["Knowledge id"].to_list()[i]))
#             #print(entire_version_updated["Similar question"].to_list()[i])
#             #print(entire_version_updated["Knowledge id"].to_list()[i])
#     df_SQ = pd.DataFrame(list(zip(similar_questions,knowledge_id_list)), columns=['Query','Label'])
#     df_SQ_all = df_SQ.explode('Query')
#     df_SQ_count = df_SQ_all["Query"].value_counts().to_frame().reset_index().rename(columns={"index":"Query","Query":"Count"})
#     df_merged = pd.merge(df_SQ_count, df_SQ_all, how='left', on = 'Query')
#     df_merged.drop_duplicates("Query",inplace=True)
#     df_duplicate_SQ = df_merged[df_merged['Count']>1]/
#     return df_SQ_all, df_duplicate_SQ

# def files_to_dataframe(published_KT_file,disabled_KT_file):
    
#     with warnings.catch_warnings(record=True):
#         warnings.simplefilter("always")
#         df_published_entire = pd.read_excel(published_KT_file,sheet_name = "entire version",header=0,engine ="openpyxl")
#         df_disabled_entire = pd.read_excel(disabled_KT_file,sheet_name = "entire version",header=0,engine ="openpyxl")  
    
#     df_published_entire['State'] = 'Published'
#     df_disabled_entire['State'] = 'Disabled'
#     entire_version_updated = pd.concat([df_published_entire,df_disabled_entire], axis=0) ##Need to change
#     entire_version_updated['Language'] = entire_version_updated['Category'].apply(lambda x: x.split('-')[0].strip())
    
#     return entire_version_updated


In [80]:
# ## Change the file name here"
# published_KT_file = r'KB\2022_06290629_P_PM.xlsx'
# disabled_KT_file = r'KB\2022_06290629_D_SKT.xlsx'
# # Change the file name here

# entire_version_updated = files_to_dataframe(published_KT_file,disabled_KT_file)
# df_SQ_all,df_duplicate_SQ = check_for_duplicate_SQ(entire_version_updated)
# df_duplicate_SQ.to_excel('Repeated_SQ_List.xlsx')
# print("File for repeated SQs created.")